In [ ]:
!pip install sentence-transformers faiss-cpu pypdf requests tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 22.0 MB/s eta 0:00:00


In [ ]:
import requests
from pathlib import Path
from pypdf import PdfReader

# Download the given ebook PDF
PDF_URL = "https://konverge.ai/pdf/Ebook-Agentic-AI.pdf"
pdf_path = Path("agentic_ai.pdf")

if not pdf_path.exists():
    print("📥 Downloading PDF...")
    pdf_path.write_bytes(requests.get(PDF_URL).content)
    print("Downloaded successfully!")
else:
    print("PDF already exists")

# Extract text from PDF
def extract_pdf_text(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        txt = page.extract_text()
        if txt:
            text += txt + "\n"
    return text

raw_text = extract_pdf_text("agentic_ai.pdf")
print("Total characters extracted:", len(raw_text))
print(raw_text[:1000])  # preview


📥 Downloading PDF...
Downloaded successfully!
Total characters extracted: 79102
Konverge AI is a decision science firm empowering 
businesses with the transformative power of AI. 
Operating at the intersection of data, machine 
learning (ML) models, and business insights, we 
help organizations develop cutting-edge AI 
products and solutions. 
This book provides actionable insights into Agentic 
AI, combining Konverge AI’s expertise with 
contributions from Emergence AI. Emergence AI 
shared its deep knowledge in autonomous 
multi-agent orchestration, addressing challenges 
like outdated systems, complex processes, and 
regulatory compliance. Their solutions enhance 
adaptability and optimize operations.
By merging these perspectives, this book is a 
practical guide to leveraging Agentic AI, helping 
businesses navigate and excel in a dynamic world.

Agentic AI
An Executive's Guide to In-depth
Understanding of Agentic AI
Preface
Artificial intelligence has transitioned from standalone 

In [ ]:
import re

def clean_text(s):
    return re.sub(r"\s+", " ", s).strip()

# Chunk Settings
CHUNK_SIZE = 800
OVERLAP = 150

def create_chunks(text):
    text = clean_text(text)
    chunks = []
    start = 0
    while start < len(text):
        end = start + CHUNK_SIZE
        chunk = text[start:end]
        chunks.append(chunk)
        start += CHUNK_SIZE - OVERLAP
    return chunks

chunks = create_chunks(raw_text)

print("Total chunks created:", len(chunks))
print("First chunk preview:\n")
print(chunks[0][:500])


Total chunks created: 118
First chunk preview:

Konverge AI is a decision science firm empowering businesses with the transformative power of AI. Operating at the intersection of data, machine learning (ML) models, and business insights, we help organizations develop cutting-edge AI products and solutions. This book provides actionable insights into Agentic AI, combining Konverge AI’s expertise with contributions from Emergence AI. Emergence AI shared its deep knowledge in autonomous multi-agent orchestration, addressing challenges like outda


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load local embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")  # Fast + lightweight

print("🔄 Creating embeddings for chunks...")
embeddings = model.encode(chunks, batch_size=16, convert_to_numpy=True, show_progress_bar=True)

embeddings = embeddings.astype("float32")
print("\n✅ Embeddings created!")
print("Shape:", embeddings.shape)   # should show (118, 384) approx


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔄 Creating embeddings for chunks...


Batches:   0%|          | 0/8 [00:00<?, ?it/s]


✅ Embeddings created!
Shape: (118, 384)


In [ ]:
# Normalize embeddings for cosine similarity
faiss.normalize_L2(embeddings)

index = faiss.IndexFlatIP(embeddings.shape[1])  # IP + normalized = cosine similarity
index.add(embeddings)

print("📌 FAISS index created!")
print("Total vectors in index:", index.ntotal)


📌 FAISS index created!
Total vectors in index: 118


In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key (sk-...): ").strip()
    print("✅ OpenAI key set.")
else:
    print("✅ OpenAI key already set.")


Enter your OpenAI API key (sk-...): ··········
✅ OpenAI key set.


In [ ]:
import requests

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

def openai_chat(messages, model="gpt-4o-mini"):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json",
    }
    data = {
        "model": model,
        "messages": messages,
        "temperature": 0.0,
        "max_tokens": 400,
    }
    resp = requests.post(url, headers=headers, json=data)
    resp.raise_for_status()
    out = resp.json()
    return out["choices"][0]["message"]["content"]


In [ ]:
def retrieve_local(query: str, top_k: int = 4):
    # Encode query using same local model
    q_emb = model.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    scores, idxs = index.search(q_emb, top_k)

    hits = []
    for score, idx in zip(scores[0], idxs[0]):
        hits.append({
            "score": float(score),
            "text": chunks[idx],
            "chunk_id": int(idx),
        })
    return hits

# quick test
test_hits = retrieve_local("What is Agentic AI?", top_k=3)
for h in test_hits:
    print("score:", round(h["score"], 4), "chunk_id:", h["chunk_id"])
    print(h["text"][:200], "\n---\n")


score: 0.8204 chunk_id: 4
ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabiliti 
---

score: 0.7635 chunk_id: 22
OF AN AGENTIC AI SYSTEM 11 A Journey into the Heart of Autonomous Intelligence Agentic AI refers to systems capable of autonomous decision-making and action in pursuit of specific objectives. We have  
---

score: 0.724 chunk_id: 5
 doubts remain. Is it truly real? Are all the claims about it accurate? In this section, we cover the following topics: � What is Agentic AI? � How does it stand apart from other AI ,What can it do? � 
---



In [ ]:
def rag_local_chat(question: str, top_k: int = 4):
    hits = retrieve_local(question, top_k=top_k)
    contexts = [h["text"] for h in hits]

    context_str = "\n\n".join(
        f"CHUNK {i+1}:\n{c}" for i, c in enumerate(contexts)
    )

    system_msg = (
        "You are a strict RAG assistant. "
        "You MUST answer only using the provided CONTEXT. "
        "If the answer is not present, reply exactly: "
        "'I don't know based on the provided document.'"
    )

    user_msg = f"CONTEXT:\n{context_str}\n\nQUESTION:\n{question}\n\nAnswer briefly."

    answer = openai_chat(
        [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ]
    )

    if hits:
        mean_score = sum(h["score"] for h in hits) / len(hits)
        confidence = round(mean_score * 100, 2)
    else:
        confidence = 0.0

    print("🧑‍💻 Question:", question)
    print("\n🤖 Answer:\n", answer)
    print("\n📊 Confidence:", confidence, "%")
    print("\n📚 Retrieved chunks:\n")
    for i, h in enumerate(hits, start=1):
        print(f"--- Chunk {i} | id={h['chunk_id']} | score={round(h['score'],4)} ---")
        print(h["text"][:400], "\n")

    return {
        "question": question,
        "answer": answer,
        "hits": hits,
        "confidence": confidence,
    }


In [ ]:
rag_local_chat("What is Agentic AI according to the ebook?")


HTTPError: 429 Client Error: Too Many Requests for url: https://api.openai.com/v1/chat/completions

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter NEW OpenAI API Key (sk-...) : ").strip()
print("🔑 OpenAI key updated in this session.")


Enter NEW OpenAI API Key (sk-...) : ··········
🔑 OpenAI key updated in this session.


In [ ]:
rag_local_chat("What is Agentic AI according to the ebook?")


HTTPError: 429 Client Error: Too Many Requests for url: https://api.openai.com/v1/chat/completions

In [ ]:
import os
from getpass import getpass

# This will ask you to paste your Groq API key like: gsk_...
os.environ["GROQ_API_KEY"] = getpass("Enter Groq API Key (gsk-...): ").strip()
print("🔑 Groq key saved in environment.")


Enter Groq API Key (gsk-...): ··········
🔑 Groq key saved in environment.


In [ ]:
import requests

GROQ_API_KEY = os.environ["GROQ_API_KEY"]

def groq_chat(messages, model="mixtral-8x7b-32768"):
    """
    Simple wrapper to call Groq's chat completion API.
    'messages' is a list like:
    [
      {"role": "system", "content": "..."},
      {"role": "user", "content": "..."}
    ]
    """
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json",
    }
    data = {
        "model": model,
        "messages": messages,
        "temperature": 0.0,
        "max_tokens": 400,
    }

    resp = requests.post(url, headers=headers, json=data)
    resp.raise_for_status()
    out = resp.json()
    return out["choices"][0]["message"]["content"]


In [ ]:
test_reply = groq_chat(
    [
        {"role": "system", "content": "You are a friendly assistant."},
        {"role": "user", "content": "Say hi in one short sentence."}
    ]
)

print("Groq test reply:", test_reply)


HTTPError: 400 Client Error: Bad Request for url: https://api.groq.com/openai/v1/chat/completions

In [ ]:
import os, requests

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

def groq_chat(messages, model="mixtral-8x7b-32768"):
    """
    Call Groq chat completion API.
    If there's an error, we print the response so we can see what's wrong.
    """
    if not GROQ_API_KEY:
        raise RuntimeError("GROQ_API_KEY is not set in environment.")

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": 0.0,
        "max_tokens": 300,
    }

    resp = requests.post(url, headers=headers, json=payload)
    if resp.status_code != 200:
        print("❌ Groq error status:", resp.status_code)
        print("❌ Groq error body:", resp.text[:500])  # show first part only
        resp.raise_for_status()
    data = resp.json()
    return data["choices"][0]["message"]["content"]


In [ ]:
test_reply = groq_chat(
    [
        {"role": "system", "content": "You are a friendly assistant."},
        {"role": "user", "content": "Say hi in one short sentence."}
    ]
)

print("Groq test reply:", test_reply)


❌ Groq error status: 400
❌ Groq error body: {"error":{"message":"The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.","type":"invalid_request_error","code":"model_decommissioned"}}



HTTPError: 400 Client Error: Bad Request for url: https://api.groq.com/openai/v1/chat/completions

In [ ]:
import os, requests

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

def groq_chat(messages, model="llama3-8b-8192"):   # <-- changed model here
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": 0.0,
        "max_tokens": 300,
    }

    resp = requests.post(url, headers=headers, json=payload)
    if resp.status_code != 200:
        print("❌ Groq error status:", resp.status_code)
        print("❌ Groq error body:", resp.text[:500])
        resp.raise_for_status()

    data = resp.json()
    return data["choices"][0]["message"]["content"]


In [ ]:
test_reply = groq_chat(
    [
        {"role": "system", "content": "You are a friendly assistant."},
        {"role": "user", "content": "Say hi in one short sentence."}
    ]
)

print("Groq test reply:", test_reply)


❌ Groq error status: 400
❌ Groq error body: {"error":{"message":"The model `llama3-8b-8192` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.","type":"invalid_request_error","code":"model_decommissioned"}}



HTTPError: 400 Client Error: Bad Request for url: https://api.groq.com/openai/v1/chat/completions

In [ ]:
!pip install transformers accelerate torch --quiet


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"

print("🔄 Loading local LLM model:", model_name)
llm_tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("✅ Local LLM is ready!")


🔄 Loading local LLM model: facebook/bart-large-cnn


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

✅ Local LLM is ready!


In [ ]:
import torch

def generate_answer_from_context(question: str, contexts, max_new_tokens:  int = 120):
    """
    Uses local BART model to answer based on retrieved context.
    If answer isn't found, it will say so.
    """
    context_str = "\n\n".join(
        f"CONTEXT {i+1}:\n{c}" for i, c in enumerate(contexts)
    )

    prompt = (
        "Use ONLY the information in CONTEXT to answer the QUESTION. "
        "If unknown, say 'I don't know based on the document.'\n\n"
        f"{context_str}\n\nQUESTION: {question}\nANSWER:"
    )

    inputs = llm_tokenizer([prompt], return_tensors="pt", max_length=1024, truncation=True)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True
        )

    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [ ]:
def rag_local_chat_offline(question: str, top_k: int = 4):
    # Retrieve relevant chunks
    hits = retrieve_local(question, top_k)
    contexts = [h["text"] for h in hits]

    # Generate answer using local model + retrieved context
    answer = generate_answer_from_context(question, contexts)

    # Confidence estimation based on FAISS similarity
    mean_score = sum(h["score"] for h in hits) / len(hits)
    confidence = round(mean_score * 100, 2)

    print("\n🧠 QUESTION:", question)
    print("\n🤖 ANSWER:\n", answer)
    print("\n📊 CONFIDENCE:", confidence, "%")
    print("\n📚 RETRIEVED CHUNKS:\n")

    for i, h in enumerate(hits, 1):
        print(f"--- Chunk {i} (Score: {round(h['score'],4)}) ---")
        print(h["text"][:400], "\n")

    return {"answer": answer, "confidence": confidence, "chunks": hits}


In [ ]:
rag_local_chat_offline("What is Agentic AI according to the ebook?")



🧠 QUESTION: What is Agentic AI according to the ebook?

🤖 ANSWER:
 This guide offers valuable insights into the potential of Agentic AI. Through six focused chapters, we explore the journey from fundamental concepts to practical applications. We delve into the intricate anatomy ofAgentic AI systems, examine the power of multi-agent collaborations, and provide frameworks for orchestrating these sophisticated technologies.

📊 CONFIDENCE: 72.37 %

📚 RETRIEVED CHUNKS:

--- Chunk 1 (Score: 0.7922) ---
ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventor 

--- Chunk 2 (Score: 0.7146) ---
OF AN AGENTIC AI SYSTEM 11 A Journey into the Heart of Autonom

{'answer': 'This guide offers valuable insights into the potential of Agentic AI. Through six focused chapters, we explore the journey from fundamental concepts to practical applications. We delve into the intricate anatomy ofAgentic AI systems, examine the power of multi-agent collaborations, and provide frameworks for orchestrating these sophisticated technologies.',
 'confidence': 72.37,
 'chunks': [{'score': 0.792242169380188,
   'text': 'ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventory, and improve lead times." Source: McKinsey & Company In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s o

In [ ]:
rag_local_chat_offline("What are the main components of an Agentic AI system?")
rag_local_chat_offline("What business value does Agentic AI provide?")
rag_local_chat_offline("How does Agentic AI differ from traditional AI?")



🧠 QUESTION: What are the main components of an Agentic AI system?

🤖 ANSWER:
 Use ONLY the information in CONTEXT to answer the QUESTION. If unknown, say 'I don't know based on the document' Use the CONTENT to help readers understand the content of this article. In this section, we define what Agentic AI is and, more importantly, what it’s not.

📊 CONFIDENCE: 70.01 %

📚 RETRIEVED CHUNKS:

--- Chunk 1 (Score: 0.7433) ---
ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventor 

--- Chunk 2 (Score: 0.7075) ---
OF AN AGENTIC AI SYSTEM 11 A Journey into the Heart of Autonomous Intelligence Agentic AI refers to systems capable of autonomous decision-m

{'answer': 'While Agentic AI promises a shift from reactive to proactive problem-solving, doubts remain. Is it truly real? Are all the claims about it accurate? What value does it bring? How are businesses using it in the real world? Let us take you through this, breaking it down so you see not just what it is, but how it works.',
 'confidence': 71.14,
 'chunks': [{'score': 0.7846667766571045,
   'text': 'ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventory, and improve lead times." Source: McKinsey & Company In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s often misunderstood. While Agentic AI p

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-large"

print("🔄 Loading stronger model:", model_name)
llm_tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("🚀 FLAN-T5 loaded successfully!")


🔄 Loading stronger model: google/flan-t5-large


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

🚀 FLAN-T5 loaded successfully!


In [ ]:
rag_local_chat_offline("What are the main components of an Agentic AI system?")
rag_local_chat_offline("What are the main components of an Agentic AI system?")
rag_local_chat_offline("What business value does Agentic AI provide?")
rag_local_chat_offline("How does Agentic AI differ from traditional AI?")



🧠 QUESTION: What are the main components of an Agentic AI system?

🤖 ANSWER:
 I don't know based on the document

📊 CONFIDENCE: 70.01 %

📚 RETRIEVED CHUNKS:

--- Chunk 1 (Score: 0.7433) ---
ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventor 

--- Chunk 2 (Score: 0.7075) ---
OF AN AGENTIC AI SYSTEM 11 A Journey into the Heart of Autonomous Intelligence Agentic AI refers to systems capable of autonomous decision-making and action in pursuit of specific objectives. We have seen this field evolve from a set of theoretical ideas to practical systems shaping industries. If you have been in AI long enough, you know we are standing on the shoulders 

{'answer': "I don't know based on the document",
 'confidence': 71.14,
 'chunks': [{'score': 0.7846667766571045,
   'text': 'ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventory, and improve lead times." Source: McKinsey & Company In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s often misunderstood. While Agentic AI promises a shift from reactive to proactive problem-solving, doubts remain. Is it truly real? Are all the claims about it accurate? In this section, we cover the following topics: � What is Agentic AI? � How do',
   'chunk_id': 4},
  {'score': 0.7252507209777832,
   'text': " doubts r

In [ ]:
!pip install google-generativeai --quiet


In [ ]:
import google.generativeai as genai
from getpass import getpass
import os

os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API Key: ")
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

print("🔑 Gemini key configured successfully!")


Enter your Gemini API Key: ··········
🔑 Gemini key configured successfully!


In [ ]:
def gemini_generate_answer(question, contexts):
    context_text = "\n\n".join(contexts)

    prompt = f"""
Answer the question strictly using the provided context.
If answer is not found in context, reply exactly:
"I don't know based on the provided document."

CONTEXT:
{context_text}

QUESTION: {question}

ANSWER:
"""

    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)

    return response.text.strip()


In [ ]:
def rag_chat_gemini(question, top_k=4):
    hits = retrieve_local(question, top_k)
    contexts = [h["text"] for h in hits]

    answer = gemini_generate_answer(question, contexts)

    mean_score = sum(h["score"] for h in hits) / len(hits)
    confidence = round(mean_score * 100, 2)

    print("\n🧠 QUESTION:", question)
    print("\n🤖 ANSWER:\n", answer)
    print("\n📊 CONFIDENCE:", confidence, "%")

    print("\n📚 CONTEXT USED:")
    for i, h in enumerate(hits, start=1):
        print(f"\n--- Chunk {i} (score={round(h['score'],4)}) ---")
        print(h["text"][:350],"...")

    return {"answer": answer, "confidence": confidence}


In [ ]:
rag_chat_gemini("What are the main components of an Agentic AI system?")


NotFound: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
import torch

def generate_answer_from_context(question: str, contexts, max_new_tokens: int = 200):
    context_str = "\n\n".join(
        f"--- Chunk {i+1} ---\n{c}" for i, c in enumerate(contexts)
    )

    prompt = f"""
You are a helpful assistant. You are given some context from a book about Agentic AI.
Use ONLY this context to answer the question. If you truly cannot find the answer,
then you may say you don't know — but first try your best to use the text.

CONTEXT:
{context_str}

QUESTION: {question}

Give a clear answer in 3–5 sentences.
ANSWER:
"""

    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1500)
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True
        )

    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [ ]:
rag_local_chat_offline("What are the main components of an Agentic AI system?", top_k=2)



🧠 QUESTION: What are the main components of an Agentic AI system?

🤖 ANSWER:
 autonomous decision-making and action in pursuit of specific objectives

📊 CONFIDENCE: 72.54 %

📚 RETRIEVED CHUNKS:

--- Chunk 1 (Score: 0.7433) ---
ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventor 

--- Chunk 2 (Score: 0.7075) ---
OF AN AGENTIC AI SYSTEM 11 A Journey into the Heart of Autonomous Intelligence Agentic AI refers to systems capable of autonomous decision-making and action in pursuit of specific objectives. We have seen this field evolve from a set of theoretical ideas to practical systems shaping industries. If you have been in AI long enough, you k

{'answer': 'autonomous decision-making and action in pursuit of specific objectives',
 'confidence': 72.54,
 'chunks': [{'score': 0.7433099746704102,
   'text': 'ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventory, and improve lead times." Source: McKinsey & Company In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s often misunderstood. While Agentic AI promises a shift from reactive to proactive problem-solving, doubts remain. Is it truly real? Are all the claims about it accurate? In this section, we cover the following topics: � What is Agentic AI? � How do',
   'chunk_id': 4},
  {'score': 0.70

In [ ]:
rag_local_chat_offline("What are the main components of an Agentic AI system?", top_k=2)
rag_local_chat_offline("What business value does Agentic AI provide?", top_k=2)
rag_local_chat_offline("How does Agentic AI differ from traditional AI?", top_k=2)



🧠 QUESTION: What are the main components of an Agentic AI system?

🤖 ANSWER:
 autonomous decision-making and action in pursuit of specific objectives

📊 CONFIDENCE: 72.54 %

📚 RETRIEVED CHUNKS:

--- Chunk 1 (Score: 0.7433) ---
ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventor 

--- Chunk 2 (Score: 0.7075) ---
OF AN AGENTIC AI SYSTEM 11 A Journey into the Heart of Autonomous Intelligence Agentic AI refers to systems capable of autonomous decision-making and action in pursuit of specific objectives. We have seen this field evolve from a set of theoretical ideas to practical systems shaping industries. If you have been in AI long enough, you k

{'answer': 'agents can produce high-quality content, reducing review cycle times by 20 to 60 percent',
 'confidence': 75.5,
 'chunks': [{'score': 0.7846667766571045,
   'text': 'ntroduction to Agentic AI 02 Anatomy of an Agentic AI System "Agentic AI going to be most of what we do in the future." "Companies need a real commitment to building AI trust and governance capabilities." "Unlike simpler gen AI architectures, agents can produce high-quality content, reducing review cycle times by 20 to 60 percent." "AI can decrease forecasting errors by 50%, reduce excess inventory, and improve lead times." Source: McKinsey & Company In this section, we will define what Agentic AI is and, more importantly, what it’s not, as it’s often misunderstood. While Agentic AI promises a shift from reactive to proactive problem-solving, doubts remain. Is it truly real? Are all the claims about it accurate? In this section, we cover the following topics: � What is Agentic AI? � How do',
   'chunk_id': 4},
